# 6.30.2025
phoneme-level mixing augmentation
음소 단위 mixing 오그멘테이션

1) select DataFrame to generate (wdf) 단어 레벨 데이터셋의 human_text 칼럼을 mixing 해서 generation 합니다.
2) 초성 종성 구분을 위해 human_text 를 초성 종성 구분하여 나누는 함수 decompose 를 불러 generation 할때 초성 종성을 각각 따로 가져옵니다.

In [104]:
import pandas as pd
from jamo import hangul_to_jamo
import hangul_jamo
import numpy as np
import soundfile as sf
import os
import librosa
from tqdm import tqdm
from itertools import chain
import pathlib

In [105]:
fold = 0

In [106]:
# h2j 로 초성, 중성, 종성 구분있게 decompose 하기
# return text in its decomposed string with chosung, jungsung and jongsung

def decompose(text):
    jamos = list(hangul_to_jamo(text))

    # Prevent re-composition by adding | token in before, in-between, and after
    decomposed = "|".join(jamos)
    decomposed = "|" +decomposed + "|"
    
    # print(f'decompose: {decomposed}')
        
    return decomposed


In [127]:
decompose('ㄸ딸기')

'|ㄸ|ᄄ|ᅡ|ᆯ|ᄀ|ᅵ|'

In [129]:
ord('ᄄ')

4356

In [107]:
# 단어 레벨 데이터셋 로딩 (생성할 텍스트 칼럼이 있는 CSV)
# orig_data = f'/data/selinawisco/kochild/nas_data/five_fold_datasets/test_fold_{fold}.csv'  # 트레인 셋 전체
orig_data = f'/home/selinawisco/children_ssd_detection/augmentation/mixing_and_others/train_1_2_3_4_error_gen_fixed.csv'
# orig_data = f'/data/selinawisco/kochild/nas_data/five_fold_datasets/test_fold_{fold}.csv'  
# orig_data = f'/home/selinawisco/whisper_evals/whisper-small-fold{fold}-42-eval.csv' # transribed test file (fold_0_test 전사된 파일)

wdf = pd.read_csv(orig_data)
wdf['human_h2j'] =wdf['human_text'].apply(decompose)
wdf['target_h2j'] =wdf['target_text'].apply(decompose)
# wdf['asr_h2j'] =wdf['asr_human_transcription'].apply(decompose)



In [108]:
len(wdf)

3665

In [109]:
# forced alignment 가 된 음소단위 데이터셋 불러오기 (학습데이터로 정렬한 데이터 !!)
DATA = f"/data/selinawisco/jamo_verified_folds_1_to_4/jamo_verified_fold_1_2_3_4.csv"
# DATA = f"/data/selinawisco/kochild/forced_aligned/fold_0/human-aligned-fold-0-train/human_aligned_fold_{fold}_train.csv"

In [110]:
df = pd.read_csv(DATA)

In [111]:
fa_vocab =set(df['fa_phoneme_label'].unique())
len(fa_vocab)

33

In [112]:
# 종성 초성 구분있게 다시 분리하기 위해서 다시 조합
df['human_text'] = df['human_text_jamo'].apply(hangul_jamo.compose)
df['target_text'] = df['target_text_jamo'].apply(hangul_jamo.compose)


In [113]:
df['human_h2j'] =df['human_text'].apply(decompose)
df['target_h2j'] =df['target_text'].apply(decompose)

In [114]:
# 초성 종성 구분 전 vocab 수:
vocab1= set()
for text in df['human_text_jamo']:
    vocab1.update(jamo for jamo in text)
fa_vocab^vocab1

{'[', ']', 'ㅄ', 'ㅒ', 'ㅖ', 'ㅙ', 'ㅚ', 'ㅢ', 'ㅣ'}

In [115]:
# 구분 후 vocab 수
vocab = set()
for text in df['human_h2j']:
    vocab.update(jamo for jamo in text)
fa_vocab ^ vocab

{'[',
 ']',
 '|',
 'ᄀ',
 'ᄁ',
 'ᄂ',
 'ᄃ',
 'ᄄ',
 'ᄅ',
 'ᄆ',
 'ᄇ',
 'ᄈ',
 'ᄉ',
 'ᄊ',
 'ᄋ',
 'ᄌ',
 'ᄍ',
 'ᄎ',
 'ᄏ',
 'ᄐ',
 'ᄑ',
 'ᄒ',
 'ᅡ',
 'ᅢ',
 'ᅣ',
 'ᅤ',
 'ᅥ',
 'ᅦ',
 'ᅧ',
 'ᅨ',
 'ᅩ',
 'ᅪ',
 'ᅫ',
 'ᅬ',
 'ᅭ',
 'ᅮ',
 'ᅯ',
 'ᅱ',
 'ᅲ',
 'ᅳ',
 'ᅴ',
 'ᅵ',
 'ᆨ',
 'ᆩ',
 'ᆫ',
 'ᆮ',
 'ᆯ',
 'ᆷ',
 'ᆸ',
 'ᆹ',
 'ᆺ',
 'ᆼ',
 'ᆽ',
 'ᆾ',
 'ㄱ',
 'ㄲ',
 'ㄴ',
 'ㄷ',
 'ㄸ',
 'ㄹ',
 'ㅁ',
 'ㅂ',
 'ㅃ',
 'ㅅ',
 'ㅆ',
 'ㅇ',
 'ㅈ',
 'ㅉ',
 'ㅊ',
 'ㅋ',
 'ㅌ',
 'ㅍ',
 'ㅎ',
 'ㅏ',
 'ㅐ',
 'ㅑ',
 'ㅓ',
 'ㅔ',
 'ㅕ',
 'ㅗ',
 'ㅘ',
 'ㅛ',
 'ㅜ',
 'ㅝ',
 'ㅟ',
 'ㅠ',
 'ㅡ'}

In [116]:
len(df)

46678

In [117]:
df.columns

Index(['Unnamed: 0', 'orig_audio', 'segment_audio', 'fa_phoneme_label',
       'phoneme_idx', 'target_phoneme', 'human_phoneme', 'start_sample',
       'end_sample', 'human_text_jamo', 'target_text_jamo', 'disease_type',
       'age', 'gender', 'id', 'subgroup', 'phoneme_pred', 'human_text',
       'target_text', 'human_h2j', 'target_h2j'],
      dtype='object')

In [118]:
# forced alignment phoneme label 이 중성 종성 초성 구분되게 다시 매핑

def jamo_at_index(text, idx):

    corrected_idx = 1 + idx * 2
    decomposed = decompose(text)
    # print("at index", corrected_idx, ": ", decomposed[corrected_idx])
    return decomposed[corrected_idx]

print(df['fa_phoneme_label'].nunique())
df['fa_phoneme_label_2'] = df.apply(lambda row: jamo_at_index(row['human_text'], row['phoneme_idx']), axis=1)



33


In [119]:
df['fa_phoneme_label_2'].nunique()

43

In [ ]:
# def jamo_from_decomposed(decomposed, orig_idx):
#     corrected_idx= 1+orig_idx*2
#     return decomposed[corrected_idx]

# df['fa_phoneme_label_2'] = df.apply(lambda row: jamo_from_decomposed(row['human_h2j'], row['phoneme_idx']), axis=1)
# s2 = set(df['fa_phoneme_label_2'].unique())

In [135]:
len(s2)
len(s2-fa_vocab)

43

In [120]:
def phoneme_mixing(df, input_text, target_text, output_dir):
    os.makedirs(output_dir, exist_ok=True)
    
    # 셔플
    np.random.seed(1)
    df = df.sample(frac=1)
    input_orig = input_text # 분리전 텍스트 저장
    input_text = decompose(input_text.strip()) # 초종성 유지하며 분리
    sdf = df[df['target_text']==target_text] # 타겟 단어만 필터 
    
    mixed_data = []
    
    jamo_paths = [] # list of phoneme paths to concatenate
    jamo_audios = []
    
    for i in range(1,len(input_text),2):
        
        jamo = input_text[i]
        # print(jamo)
        candidates = sdf[sdf['fa_phoneme_label_2']==jamo] # 먼저 타겟단어 안에서 자모 가져오기
        if len(candidates) > 1: 
            jamo_path = candidates.sample(n=1).iloc[0]['segment_audio']
            jamo_audio, sr = librosa.load(jamo_path, sr=16000)
            
            jamo_paths.append(jamo_path)
            jamo_audios.append(jamo_audio)
        else: # 해당 타겟단어안에서 음소가 없으면 전체에서 가져오기
            # print("jamo not found in target text")
            all_candidates = df[df['fa_phoneme_label_2']==jamo]
            # print(ord(df[df['fa_phoneme_label']]))
            # print(ord(jamo))
            if len(all_candidates) == 0:
                # Skip this input_text because one phoneme is missing
                print(f"Skipping '{input_orig}' — no match for jamo '{jamo}({ord(jamo)})'")
                return []  # or continue to next input_text in a loop, if applicable

            jamo_path = all_candidates.sample(n=1).iloc[0]['segment_audio'] 
            jamo_paths.append(jamo_path)
            
            jamo_audio, sr = librosa.load(jamo_path, sr=16000)
            jamo_audios.append(jamo_audio)
            
    output = np.concatenate(jamo_audios)
    output_path = os.path.join(output_dir, f"phoneme_mixing_{input_text}.wav")
    sf.write(output_path, output, sr)
    
    # csv entries
    mixed_data.append({
        "audio": output_path,
        "target_text": target_text,
        "human_text": input_orig,
        "human_text_decomposed": input_text,
    })
    
    return mixed_data
    

In [22]:
def phoneme_mixing_age(df, input_text, target_text, age, output_dir):
    os.makedirs(output_dir, exist_ok=True)
    
    # 셔플
    np.random.seed(1)
    df = df.sample(frac=1)
    input_orig = input_text # 분리전 텍스트 저장
    input_text = decompose(input_text.strip()) # 초종성 유지하며 분리
    sdf = df[df['target_text']==target_text] # 타겟 단어만 필터 
    
    mixed_data = []
    
    jamo_paths = [] # list of phoneme paths to concatenate
    jamo_audios = []
    
    for i in range(1,len(input_text),2):
        
        jamo = input_text[i]
        # print(jamo)
        jamo_candidates = sdf[sdf['fa_phoneme_label_2']==jamo] # 먼저 타겟단어 안에서 자모 가져오기
        candidates = jamo_candidates[jamo_candidates['age']==age]
        
        if len(candidates) > 1: 
            jamo_path = candidates.sample(n=1).iloc[0]['segment_audio']
            jamo_audio, sr = librosa.load(jamo_path, sr=16000)
            
            jamo_paths.append(jamo_path)
            jamo_audios.append(jamo_audio)
        else: # 해당 타겟단어안에서 음소가 없으면 전체에서 가져오기
            # print("jamo not found in target text")
            all_jamo_candidates = df[df['fa_phoneme_label_2']==jamo]
            all_candidates = all_jamo_candidates[all_jamo_candidates['age']==age]
            
            if len(all_candidates) == 0:
                # Skip this input_text because one phoneme is missing
                print(f"Skipping '{input_orig}' — no match for jamo '{jamo}({ord(jamo)})'")
                return []  # or continue to next input_text in a loop, if applicable

            jamo_path = all_candidates.sample(n=1).iloc[0]['segment_audio'] 
            jamo_paths.append(jamo_path)
            
            jamo_audio, sr = librosa.load(jamo_path, sr=16000)
            jamo_audios.append(jamo_audio)
            
    output = np.concatenate(jamo_audios)
    output_path = os.path.join(output_dir, f"phoneme_mixing_{input_text}.wav")
    sf.write(output_path, output, sr)
    
    # csv entries
    mixed_data.append({
        "audio": output_path,
        "target_text": target_text,
        "human_text": input_orig,
        "human_text_decomposed": input_text,
    })
    
    return mixed_data
    

나이 안에서 믹싱

In [ ]:
# 나이 고려

# # 발음 틀린 음성만 생성하기
swdf = wdf[wdf['new_label']==1]

# wdf = wdf.head(5) 
tqdm.pandas()

csv_used_name = pathlib.Path(orig_data).stem
print("CSV used to generate human_texts (word level) ", csv_used_name)

print("phoneme segments used to mix are from ",pathlib.Path(DATA) )
phonemes_from = pathlib.Path(DATA).stem

# 음성 저장할 경로
output_dir = f"/data/selinawisco/phoneme_mixing/mixed_{csv_used_name}_from_{phonemes_from}_age--"
mixed_data_all = list(chain.from_iterable(
    swdf.progress_apply(lambda row: phoneme_mixing_age(df, row['human_text'], row['target_text'], row['age'], output_dir=output_dir), axis=1)
))
print("mixed audio saved to ", output_dir)

mixed_df = pd.DataFrame(mixed_data_all)
mixed_df.to_csv(f'{output_dir}/{csv_used_name}_age.csv', index=False)

print("mixed audio CSV saved to ", f'{output_dir}/{csv_used_name}_from {phonemes_from}.csv')

오발음만 생성

In [ ]:
# # 발음 틀린 음성만 생성하기
wdf = wdf[wdf['new_label']==1]

# wdf = wdf.head(5) 
tqdm.pandas()

csv_used_name = pathlib.Path(orig_data).stem
print("CSV used to generate human_texts (word level) ", csv_used_name)

print("phoneme segments used to mix are from ",pathlib.Path(DATA) )
phonemes_from = pathlib.Path(DATA).stem

# 음성 저장할 경로
output_dir = f"/data/selinawisco/phoneme_mixing/mixed_{csv_used_name}_from_verified_{phonemes_from}_fixed"
mixed_data_all = list(chain.from_iterable(
    wdf.progress_apply(lambda row: phoneme_mixing(df, row['human_text'], row['target_text'], output_dir=output_dir), axis=1)
))
print("mixed audio saved to ", output_dir)

mixed_df = pd.DataFrame(mixed_data_all)
mixed_df.to_csv(f'{output_dir}/{csv_used_name}.csv', index=False)

print("mixed audio CSV saved to ", f'{output_dir}/{csv_used_name}_from {phonemes_from}.csv')

정발음만 생성

In [ ]:
# 발음 맞춘 음성만 생성하기
wdf = wdf[wdf['new_label']==0]

# wdf = wdf.head(5) 
tqdm.pandas()

csv_used_name = pathlib.Path(orig_data).stem
print("CSV used to generate human_texts (word level) ", csv_used_name)

print("phoneme segments used to mix are from ",pathlib.Path(DATA) )

# 음성 저장할 경로
output_dir = f"/data/selinawisco/phoneme_mixing/mixed_{csv_used_name}_correct_v1"
mixed_data_all = list(chain.from_iterable(
    wdf.progress_apply(lambda row: phoneme_mixing(df, row['human_text'], row['target_text'], output_dir=output_dir), axis=1)
))
print("mixed audio saved to ", output_dir)

mixed_df = pd.DataFrame(mixed_data_all)
mixed_df.to_csv(f'{output_dir}/{csv_used_name}_correct_v1.csv', index=False)

print("mixed audio CSV saved to ", f'{output_dir}/{csv_used_name}_correct_v1.csv')

CSV used to generate human_texts (word level)  test_fold_0
phoneme segments used to mix are from  /data/selinawisco/kochild/forced_aligned/fold_0/human-aligned-fold-0-train/human_aligned_fold_0_train.csv


100%|██████████| 2587/2587 [01:35<00:00, 27.20it/s]

mixed audio saved to  /data/selinawisco/phoneme_mixing/mixed_test_fold_0_correct_v1
mixed audio CSV saved to  /data/selinawisco/phoneme_mixing/mixed_test_fold_0_correct_v1/test_fold_0_correct_v1.csv


In [ ]:
# # sample run: 데이터셋에 없는 텍스트 만들어보기
# phoneme_mixing(df, input_text="호랑이",target_text= "호랑이", output_dir="./phoneme_mixing")


[{'audio': './phoneme_mixing/phoneme_mixing_|ᄒ|ᅩ|ᄅ|ᅡ|ᆼ|ᄋ|ᅵ|.wav',
  'target_text': '호랑이',
  'human_text': '|ᄒ|ᅩ|ᄅ|ᅡ|ᆼ|ᄋ|ᅵ|'}]

기존 train file 과 병합

In [133]:
# fold 0 train + mixed 합치기
mixed = pd.read_csv(f'/data/selinawisco/phoneme_mixing/mixed_train_1_2_3_4_error_gen_fixed_from_verified_jamo_verified_fold_1_2_3_4_fixed/train_1_2_3_4_error_gen_fixed.csv')
orig_train = pd.read_csv(f'/data/selinawisco/kochild/five_fold_datasets/test_fold_{fold}_train.csv')

In [134]:
orig_train.columns

Index(['audio', 'disease_type', 'age', 'gender', 'subgroup', 'id',
       'textgrid_text', 'target_text', 'human_text', 'asr_text',
       'target_text_jamo', 'human_text_jamo', 'new_label'],
      dtype='object')

In [135]:
mixed.columns

Index(['audio', 'target_text', 'human_text', 'human_text_decomposed'], dtype='object')

In [136]:
orig_train = orig_train.drop(columns=['disease_type', 'age', 'gender', 'subgroup', 'id',
       'textgrid_text', 'asr_text',
       'target_text_jamo', 'new_label'])

In [137]:
mixed = mixed.drop(columns=["human_text_decomposed"])
mixed['human_text_jamo'] = mixed['human_text'].apply(hangul_jamo.decompose)

In [138]:
mixed

,audio,target_text,human_text,human_text_jamo
0,/data/selinawisco/phoneme_mixing/mixed_train_1...,눈사람,눈사암,ㄴㅜㄴㅅㅏㅇㅏㅁ
1,/data/selinawisco/phoneme_mixing/mixed_train_1...,눈사람,눈다암,ㄴㅜㄴㄷㅏㅇㅏㅁ
2,/data/selinawisco/phoneme_mixing/mixed_train_1...,장갑,상갑,ㅅㅏㅇㄱㅏㅂ
3,/data/selinawisco/phoneme_mixing/mixed_train_1...,눈사람,눈사란,ㄴㅜㄴㅅㅏㄹㅏㄴ
4,/data/selinawisco/phoneme_mixing/mixed_train_1...,눈사람,눈따란,ㄴㅜㄴㄸㅏㄹㅏㄴ
...,...,...,...,...
2117,/data/selinawisco/phoneme_mixing/mixed_train_1...,눈사람,눈타랑,ㄴㅜㄴㅌㅏㄹㅏㅇ
2118,/data/selinawisco/phoneme_mixing/mixed_train_1...,햄버거,햄러거,ㅎㅐㅁㄹㅓㄱㅓ
2119,/data/selinawisco/phoneme_mixing/mixed_train_1...,바퀴,빠퀴,ㅃㅏㅋㅟ
2120,/data/selinawisco/phoneme_mixing/mixed_train_1...,바퀴,빠휘,ㅃㅏㅎㅟ


In [139]:
orig_train

,audio,target_text,human_text,human_text_jamo
0,/data/selinawisco/kochild/APAC/일반아동/일반_clear/1...,빨대,빨대,ㅃㅏㄹㄷㅐ
1,/data/selinawisco/kochild/APAC/일반아동/일반_clear/1...,사탕,사탕,ㅅㅏㅌㅏㅇ
2,/data/selinawisco/kochild/APAC/일반아동/일반_clear/1...,침대,침대,ㅊㅣㅁㄷㅐ
3,/data/selinawisco/kochild/APAC/일반아동/일반_clear/1...,꽃,꽃,ㄲㅗㅊ
4,/data/selinawisco/kochild/APAC/일반아동/일반_clear/1...,바퀴,바퀴,ㅂㅏㅋㅟ
...,...,...,...,...
16939,/data/selinawisco/kochild/K_APP/구개열아동/KAPP낱말/2...,눈,뉴,ㄴㅠ
16940,/data/selinawisco/kochild/K_APP/구개열아동/KAPP낱말/2...,쨈,떄이,ㄸㅒㅇㅣ
16941,/data/selinawisco/kochild/K_APP/구개열아동/KAPP낱말/2...,쨈,떄이,ㄸㅒㅇㅣ
16942,/data/selinawisco/kochild/K_APP/구개열아동/KAPP낱말/2...,총,툥,ㅌㅛㅇ


In [140]:
augmented = pd.concat([orig_train, mixed])
augmented # 1464 + 16944



,audio,target_text,human_text,human_text_jamo
0,/data/selinawisco/kochild/APAC/일반아동/일반_clear/1...,빨대,빨대,ㅃㅏㄹㄷㅐ
1,/data/selinawisco/kochild/APAC/일반아동/일반_clear/1...,사탕,사탕,ㅅㅏㅌㅏㅇ
2,/data/selinawisco/kochild/APAC/일반아동/일반_clear/1...,침대,침대,ㅊㅣㅁㄷㅐ
3,/data/selinawisco/kochild/APAC/일반아동/일반_clear/1...,꽃,꽃,ㄲㅗㅊ
4,/data/selinawisco/kochild/APAC/일반아동/일반_clear/1...,바퀴,바퀴,ㅂㅏㅋㅟ
...,...,...,...,...
2117,/data/selinawisco/phoneme_mixing/mixed_train_1...,눈사람,눈타랑,ㄴㅜㄴㅌㅏㄹㅏㅇ
2118,/data/selinawisco/phoneme_mixing/mixed_train_1...,햄버거,햄러거,ㅎㅐㅁㄹㅓㄱㅓ
2119,/data/selinawisco/phoneme_mixing/mixed_train_1...,바퀴,빠퀴,ㅃㅏㅋㅟ
2120,/data/selinawisco/phoneme_mixing/mixed_train_1...,바퀴,빠휘,ㅃㅏㅎㅟ


In [141]:
augmented.to_csv('/data/selinawisco/kochild/five_fold_datasets_with_phoneme_mixing/train_fold_0_and_train_1_2_3_4_error_gen_fixed.csv', index=False)


정발음 믹싱음성도 불러와 같이 병합

In [131]:
#정상음성도 포함
mdf = pd.read_csv('/data/selinawisco/phoneme_mixing/mixed_train_1_2_3_4_error_gen_fixed_from_verified_jamo_verified_fold_1_2_3_4_fixed/train_1_2_3_4_error_gen_fixed.csv')
mdf = mdf.drop(columns=["human_text_decomposed"])
mdf['human_text_jamo'] = mdf['human_text'].apply(hangul_jamo.decompose)
cdf = pd.read_csv('/data/selinawisco/phoneme_mixing/mixed_test_fold_0_correct_v1/test_fold_0_correct_v1.csv')
cdf = cdf.drop(columns=["human_text_decomposed"])
cdf['human_text_jamo'] = cdf['human_text'].apply(hangul_jamo.decompose)

result = pd.concat([mdf,cdf])
result_final = pd.concat([orig_train,result])
print(len(result_final))
result_final.to_csv('/data/selinawisco/phoneme_mixing/train_fold_0_and_mixed_train_1_2_3_4_error_gen_fixed_verified_and_correct_v1.csv', index=False)


21653
